In [ ]:
#Instalar conector a base de datos si es necesario
!pip install psycopg2-binary sqlalchemy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.5 MB/s eta 0:00:00


In [ ]:
#Conectarse a base de datos
import psycopg2

# credenciales
host = "dpg-d2cgloidbo4c73boagu0-a.oregon-postgres.render.com"
database = "gestoresrcd"
user = "tacevedo"
password = "YuSCfgvhBUbj83qi0Rxysegik5jd0kx8"
port = "5432"

# conexión
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

# probar consulta
cursor = conn.cursor()

In [ ]:
# cargar archivos sql a la base de datos
with open("gestoresRCD.sql", "r", encoding="utf-8") as f:
    sql_code = f.read()
    cursor.execute(sql_code)

with open("itemsEjec.sql", "r", encoding="utf-8") as f:
    sql_code = f.read()
    cursor.execute(sql_code)
conn.commit()

In [ ]:
# Listar tablas en la base de datos
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")
tablas = cursor.fetchall()
print("Tablas en la base de datos:")
for t in tablas:
    print("-", t[0])


Tablas en la base de datos:
- gestoresrcd
- itemsejec


In [ ]:
# breve visualización de las tablas
cursor.execute("SELECT * FROM gestoresrcd LIMIT 10;")
for fila in cursor.fetchall():
    print(fila)

print("")

cursor.execute("SELECT * FROM itemsejec LIMIT 10;")
for fila in cursor.fetchall():
    print(fila)

(1, 'RECICLADOS INDUSTRIALES DE COLOMBIA S.A.S.', '900360954-1', '3168766867', 'comercial.antioquia@recicladosindustriales.com.co', 'https://www.recicladosindustriales.co', 'GIRARDOTA', 'Vereda Portachuelo, predio Reciclados Industriales de Colombia S.A.S.', Decimal('-75.4725553321950'), Decimal('6.37855551443530'), 'VALLE DE ABURRA', 'APROVECHAMIENTO', 'CORANTIOQUIA', 'PvF', 'No se paga, ni se cobra', 'Se reciben materiales pétreos previamente clasificados, destinados a la producción de base granular, subase y fresado de pavimento; adicionalmente, se ofrece pago por el concreto limpio entregado.')
(2, 'RECICLADOS INDUSTRIALES DE COLOMBIA S.A.S.', '900360954-1', '3168766867', 'comercial.antioquia@recicladosindustriales.com.co', 'https://www.recicladosindustriales.co', 'GIRARDOTA', 'Vereda Portachuelo, predio Reciclados Industriales de Colombia S.A.S.', Decimal('-75.4725553321950'), Decimal('6.37855551443530'), 'VALLE DE ABURRA', 'APROVECHAMIENTO', 'CORANTIOQUIA', 'Concreto', 'Se paga p

In [ ]:
#cerrar conexión base de datos
cursor.close()
conn.close()

In [ ]:
!pip install folium


In [ ]:
# Instalar dependencias
!pip install ipyleaflet ipywidgets --quiet

from ipyleaflet import Map, Marker
from ipywidgets import Output

# Centro de Antioquia
center_lat, center_lon = 6.5569, -75.8281

# Crear mapa
m = Map(center=(center_lat, center_lon), zoom=8)

# Variable para guardar coordenadas
selected_coords = {}

# Función para manejar click
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')
        selected_coords["lat"] = lat
        selected_coords["lon"] = lon
        print(f"Coordenadas guardadas: {lat}, {lon}")
        # Agregar marcador
        m.layers = m.layers[:1]  # Quita marcadores anteriores
        m.add_layer(Marker(location=(lat, lon)))

# Vincular evento
m.on_interaction(handle_click)

# Mostrar mapa
m


Map(center=[6.5569, -75.8281], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Coordenadas guardadas: 6.588211687722278, -75.82484671050875
Coordenadas guardadas: 6.773694930861475, -76.6474699096419
Coordenadas guardadas: 6.773694930861475, -76.6474699096419
Coordenadas guardadas: 6.607305103698831, -76.27937128732918
Coordenadas guardadas: 6.68232872886886, -76.22099573028139
Coordenadas guardadas: 6.71301690231645, -76.38513406127461
Coordenadas guardadas: 6.653013331326967, -76.20117187500001


In [ ]:
!pip install ipyleaflet ipywidgets pandas shapely --quiet

import pandas as pd
from shapely.geometry import MultiPoint
from ipyleaflet import Map, Polygon, LayersControl

# Colores para cada tipo
colors = {
    "Base": "blue",
    "Concreto": "red",
    "PvF": "green",
    "Tierras": "orange"
}

# Archivos CSV
files = {
    "Base": "itemsBase.csv",
    "Concreto": "itemsConcreto.csv",
    "PvF": "itemsPvF.csv",
    "Tierras": "itemsTierras.csv"
}

# Crear mapa centrado en Antioquia
center_lat, center_lon = 6.5569, -75.8281
m = Map(center=(center_lat, center_lon), zoom=8)

def add_polygon_from_points(nombre, file, color):
    df = pd.read_csv(file)
    # Lista de coordenadas (lon, lat) para Shapely
    coords = list(zip(df['Longitud (X)'], df['Latitud (Y)']))
    multipoint = MultiPoint(coords)

    # Calcular polígono envolvente
    hull = multipoint.convex_hull

    if hull.geom_type == 'Polygon':
        # Convertir a formato (lat, lon) para ipyleaflet
        coords_latlon = [(lat, lon) for lon, lat in hull.exterior.coords]
        poly = Polygon(
            locations=coords_latlon,
            color=color,
            fill_color=color,
            fill_opacity=0.4,
            name=nombre
        )
        m.add_layer(poly)

# Agregar polígonos para cada archivo
for nombre, file in files.items():
    add_polygon_from_points(nombre, file, colors[nombre])

# Control de capas para mostrar/ocultar
m.add_control(LayersControl(position='topright'))

m


Map(center=[6.5569, -75.8281], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [3]:
# Instalar dependencias si aún no están
!pip install ipyleaflet ipywidgets --quiet

# Importar módulos
from ipyleaflet import Map, Marker
from ipywidgets import Output, VBox, Button

# Centro de Antioquia
center_lat, center_lon = 6.5569, -75.8281

# Crear mapa
m = Map(center=(center_lat, center_lon), zoom=8)

# Variable para guardar coordenadas
selected_coords = {}

# Salida para mostrar mensajes
out = Output()

# Función para manejar clic en el mapa
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')
        selected_coords["lat"] = lat
        selected_coords["lon"] = lon
        with out:
            out.clear_output()
            print(f"📍 Coordenadas guardadas: {lat:.5f}, {lon:.5f}")
        # Limpiar marcadores anteriores y agregar uno nuevo
        m.layers = m.layers[:1]
        m.add_layer(Marker(location=(lat, lon)))

# Botón para mostrar coordenadas guardadas
btn_show = Button(description="Mostrar coordenadas guardadas")
def show_coords(btn):
    with out:
        out.clear_output()
        if selected_coords:
            print(f"✅ Coordenadas actuales: {selected_coords}")
        else:
            print("❗ Aún no has seleccionado ninguna ubicación.")
btn_show.on_click(show_coords)

# Vincular el evento de clic
m.on_interaction(handle_click)

# Mostrar todo junto
VBox([m, btn_show, out])



In [4]:
# Instalar dependencias si aún no están
!pip install ipyleaflet ipywidgets --quiet

# Importar módulos
from ipyleaflet import Map, Marker, Polygon
from ipywidgets import Output, VBox, Button

# Centro de Antioquia
center_lat, center_lon = 6.5569, -75.8281

# Crear mapa
m = Map(center=(center_lat, center_lon), zoom=8)

# Variable para guardar coordenadas
selected_coords = {}

# Salida para mensajes
out = Output()

# Función para manejar clic en el mapa
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')
        selected_coords["lat"] = lat
        selected_coords["lon"] = lon
        with out:
            out.clear_output()
            print(f"📍 Coordenadas guardadas: {lat:.5f}, {lon:.5f}")
        # Eliminar solo los marcadores previos
        m.layers = [layer for layer in m.layers if not isinstance(layer, Marker)]
        # Agregar nuevo marcador
        m.add_layer(Marker(location=(lat, lon)))

# Botón para mostrar coordenadas guardadas
btn_show = Button(description="Mostrar coordenadas guardadas")
def show_coords(btn):
    with out:
        out.clear_output()
        if selected_coords:
            print(f"✅ Coordenadas actuales: {selected_coords}")
        else:
            print("❗ Aún no has seleccionado ninguna ubicación.")
btn_show.on_click(show_coords)

# Vincular el evento
m.on_interaction(handle_click)

# Agregar un polígono de ejemplo (puedes usar tus propias coordenadas)
poly_coords = [
    (6.8, -75.8),
    (6.7, -75.5),
    (6.5, -75.6),
    (6.6, -75.9)
]

polygon = Polygon(
    locations=poly_coords,
    color="blue",
    fill_color="blue",
    fill_opacity=0.4
)

# Agregar polígono al mapa
m.add_layer(polygon)

# Mostrar todo
VBox([m, btn_show, out])
